In [61]:
#!pip install seaborn
#!pip install openpyxl
#!pip install sklearn



In [62]:
import pandas as pd
import numpy as np
import random as rd
import time
import csv
import seaborn as sbs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [63]:

file_data = 'lynx.csv'
path_name='../datasets/'
path_name_results='../results/'
file_result = 'Result_MLP_canadian_lynx.csv'

In [64]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =',', encoding = 'latin1', decimal='.', usecols=[2])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   value   114 non-null    int64
dtypes: int64(1)
memory usage: 1.0 KB


In [65]:
dataset

,value
0,269
1,321
2,585
3,871
4,1475
...,...
109,662
110,1000
111,1590
112,2657


In [66]:
#checks if there are null variables
dataset.isna().sum()

value    0
dtype: int64

In [67]:
def salvar_resultado(nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration):
  #Script to write training cycle results
  data = [nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration]
  fields = ['Dataset','Best Params','n_time_steps','MSE','RMSE','MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['Dataset','Best Params','n_time_steps','MSE','RMSE','MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)    

In [68]:
def previsao_MLP(nm_dataset, dataset, n_time_steps):
  # dataframe tratament
  df = pd.DataFrame()
  df['value']=dataset['value']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['value'].shift(n_step)  
      
  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  80% treinam  20% test
  nlinhas = int(np.round(df.shape[0] *0.80)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 1 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values
  print('X_train')
  print(X_train.shape)
  X_test = df.iloc[nlinhas:dataset.shape[0],1: 1 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  print('X_test')
  print(X_test.shape)
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  # Cross-validated for time series
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  param_grid = {
      'hidden_layer_sizes': [(4,6,1),(2,6,1),(6,12,1),(6,18,1)], # MLP layers
      'max_iter': [ 500], # maximum iterations
      'activation': [ 'relu','identity'], # activation function
      'solver': ['adam'], # weight optimization algorithm
      'alpha': [0.0001, 0.001, 0.01],  # alpha strength of regularization
  }
  
  modelo = MLPRegressor(random_state=0)
  
  grid = GridSearchCV(modelo, param_grid, n_jobs= -1,scoring='neg_mean_absolute_percentage_error', cv=ts_cv, verbose=1)
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))  

  # Stores the training execution end time
  Hora_Fim = time.time()  

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio  

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)  
  
  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100   
  
  sMAPE = round(
		np.mean(
			np.abs(predict - y_test) /
			((np.abs(predict) + np.abs(y_test)))
		)*100, 2
	)

  salvar_resultado(nm_dataset, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duracao)

In [69]:
#create file to results
criar_arquivo_resultado()

print('forecast for canadian lynx')
for n_time_steps in range(1,13): #predict with 1 to 13 past values of medition
    grid = previsao_MLP('c.lynx', dataset, n_time_steps)

forecast for canadian lynx
X_train
(90, 1)
X_test
(23, 1)
Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['c.lynx', "{'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (6, 12, 1), 'max_iter': 500, 'solver': 'adam'}", 1, 4752793.301035415, 2180.0902047932364, 1536.6831993715223, 99.92514677946865, 99.85, 29.247299909591675]
X_train
(90, 2)
X_test
(22, 2)
Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['c.lynx', "{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 2, 820986.1526353885, 906.0828619035836, 465.0813245002596, 62.67702491857615, 37.07, 40.925472259521484]
X_train
(89, 3)
X_test
(22, 3)
Fitting 5 folds for each of 24 candidates, totalling 120 fits


...

c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['c.lynx', "{'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': (4, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 10, 985498.9552929243, 992.723000284029, 666.0165054719844, 201.97431463273244, 37.82, 36.228073835372925]
X_train
(82, 11)
X_test
(21, 11)
Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['c.lynx', "{'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 11, 4166169.666507411, 2041.1197090095943, 1131.2568692613206, 99.80992721151135, 99.62, 37.64295434951782]
X_train
(82, 12)
X_test
(20, 12)
Fitting 5 folds for each of 24 candidates, totalling 120 fits
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['c.lynx', "{'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (4, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 12, 1099246.845988314, 1048.4497346026246, 455.60681573350894, 130.00408207058683, 36.46, 39.64352607727051]


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
